In [4]:
from os import listdir
from os.path import isfile, join
mypath = "C:\Users\Christophe\Documents\GitHub\MSDA - Coursework\Data 602\Data 602 - Advanced Programming\Untitled Folder"
onlyfiles = [f for f in listdir(mypath) if isfile(join(mypath, f))]

In [5]:
onlyfiles

['epa-http.txt.Z', 'Untitled.ipynb']

In [13]:
import zlib

zlib.decompress("epa-http.txt.Z", zlib.MAX_WBITS|16)

error: Error -3 while decompressing data: incorrect header check

In [23]:
str_object1 = open('epa-http.txt.Z', 'rb').read()


''